In [14]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the CSV file
file_path = 'c:\\repos\\python\\Bacteria_counter\\runs\\detect\\608px-100_epoch-yolos-augment(BEST)\\results.csv'
data = pd.read_csv(file_path)

# Specify the column you want to plot
column_to_plot = 'train/box_loss'  # Change this to your desired column name

# Check if the column exists in the DataFrame
if column_to_plot in data.columns:
    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.plot(data['epoch'], data[column_to_plot], marker='o', linestyle='-', color='b')
    plt.xlabel('Epoch')
    plt.ylabel(column_to_plot)
    plt.title(f'{column_to_plot} over Epochs')
    plt.grid(True)
    plt.show()
else:
    print(f"Column '{column_to_plot}' not found in the CSV file.")


Column 'train/box_loss' not found in the CSV file.
